# Covid Vaccination in England by region and age group

In this notebook we'll learn how to visualise England's Covid vaccination data by region and age group.

In [1]:
import sys
sys.path.append("..")

import pandas as pd
import altair as alt
import data as dt
import numpy as np
from dateutil import parser

import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

In [2]:
latest_weekly_date = parser.parse("2021-04-22")
spreadsheet = f"../data/COVID-19-weekly-announced-vaccinations-{latest_weekly_date.strftime('%-d-%B-%Y')}.xlsx"

In [5]:
vaccinations = dt.vaccinations_dataframe(spreadsheet)    
population = dt.population_dataframe(spreadsheet)    
combined = dt.compute_all_vaccination_rates(vaccinations, population)
combined.head()

,Region Code (Administrative),Region Name (administrative),UTLA Code,UTLA Name,LTLA Code,LTLA Name,Under 45,45-49,50-54,55-59,60-64,65-69,70-74,75-79,80+
14,E12000004,East Midlands,E10000007,Derbyshire,E07000032,Amber Valley,20.710925,83.505741,90.6881,90.973661,93.232649,95.570698,96.711963,97.804796,97.197598
15,E12000004,East Midlands,E10000024,Nottinghamshire,E07000170,Ashfield,19.840826,70.251111,87.901663,90.642105,91.379967,94.311954,95.65667,96.470588,96.369114
16,E12000004,East Midlands,E10000024,Nottinghamshire,E07000171,Bassetlaw,19.617013,73.056929,87.851962,90.432421,92.098765,94.227747,96.442151,96.933875,96.920755
17,E12000004,East Midlands,E10000018,Leicestershire,E07000129,Blaby,20.70238,77.088552,89.326138,90.098742,92.064951,94.401756,96.67184,97.177513,96.244375
18,E12000004,East Midlands,E10000007,Derbyshire,E07000033,Bolsover,19.286211,72.523607,87.753264,90.084355,92.479523,93.790341,95.613048,97.542155,97.002794


In [29]:
population.loc[:, "Region Name (administrative)"] = vaccinations["Region Name (administrative)"]
population.loc[:, "UTLA Name"] = vaccinations["UTLA Name"]

population_by_region = population.groupby(["Region Name (administrative)"]).sum()
population_by_region.insert(0, "Region", list(population_by_region.index))

vaccinations_by_region = vaccinations.groupby(["Region Name (administrative)"]).sum()
vaccinations_by_region.insert(0, "Region", list(vaccinations_by_region.index))

display(population_by_region.head().style.hide_index())
display(vaccinations_by_region.head().style.hide_index())

Region,Under 45,45-49,50-54,55-59,60-64,65-69,70-74,75-79,80+
East Midlands,2844103,328863,364409,356521,304239,261855,265767,193832,249081
East of England,3726380,442911,470310,456949,387533,331644,344394,251884,346109
London,6881091,721234,665536,587360,458540,342411,293405,204674,296908
North East,1518387,170690,193328,200368,180689,154630,150299,102321,140094
North West,4458869,499372,549736,544269,465525,389133,387993,274200,364273


Region,Under 45,45-49,50-54,55-59,60-64,65-69,70-74,75-79,80+
East Midlands,485460,218006,311742,313686,274581,243581,253313,186637,239302
East of England,569351,241143,401257,401575,347677,305560,326115,241330,331956
London,853819,343168,463764,431490,354518,277006,249018,176440,256057
North East,249748,91514,166563,179486,166025,145734,144437,99398,135586
North West,726932,267231,453918,468098,413607,356795,366003,261963,347803


In [100]:
vaccination_rates_by_region = ((vaccinations_by_region
    .select_dtypes(exclude='object')
    .div(population_by_region.select_dtypes(exclude='object')) * 100)
    .combine_first(vaccinations_by_region)[vaccinations_by_region.columns])

vaccination_rates_by_region.loc[:, "Overall"] = vaccinations_by_region.sum(axis=1).div(population_by_region.sum(axis=1)) * 100
vaccination_rates_by_region = vaccination_rates_by_region.convert_dtypes()

formatting = {
    column: "{:.2f}" 
    for column in vaccination_rates_by_region.drop(["Region"], axis=1).columns
}
(vaccination_rates_by_region
 .sort_values(["Overall"], ascending=False)
 .style.format(formatting)
 .hide_index())

Region,Under 45,45-49,50-54,55-59,60-64,65-69,70-74,75-79,80+,Overall
South West,15.94,57.64,87.38,89.32,90.73,93.17,95.16,96.25,96.50,50.40
North East,16.45,53.61,86.16,89.58,91.88,94.25,96.10,97.14,96.78,49.04
East Midlands,17.07,66.29,85.55,87.99,90.25,93.02,95.31,96.29,96.07,48.88
East of England,15.28,54.45,85.32,87.88,89.72,92.13,94.69,95.81,95.91,46.85
Yorkshire and The Humber,15.76,61.11,84.55,88.09,90.25,92.85,95.19,96.25,95.94,46.78
South East,14.97,54.21,85.73,88.03,89.79,92.21,94.48,95.59,95.76,46.68
North West,16.30,53.51,82.57,86.00,88.85,91.69,94.33,95.54,95.48,46.16
West Midlands,15.73,60.61,82.70,85.57,88.65,91.54,94.20,95.29,94.52,45.83
London,12.41,47.58,69.68,73.46,77.31,80.90,84.87,86.21,86.24,32.58


In [75]:
population_by_utla = population.groupby(["UTLA Name"]).sum()
population_by_utla.insert(0, "UTLA Name", list(population_by_utla.index))

vaccinations_by_utla = vaccinations.groupby(["UTLA Name"]).sum()
vaccinations_by_utla.insert(0, "UTLA Name", list(vaccinations_by_utla.index))

display(population_by_utla.head().style.hide_index())
display(vaccinations_by_utla.head().style.hide_index())

UTLA Name,Under 45,45-49,50-54,55-59,60-64,65-69,70-74,75-79,80+
Barking and Dagenham,169657,16932,15431,12730,9587,6687,5187,3536,5379
Barnet,279120,31443,28629,25283,21045,17051,15591,11181,16774
Barnsley,137356,16141,19575,18598,16087,13670,13287,9690,11738
Bath and North East Somerset,116661,12373,13382,13299,11390,9811,10140,7535,11063
Bedford,114486,13257,13766,12838,10518,8706,8615,5980,8635


UTLA Name,Under 45,45-49,50-54,55-59,60-64,65-69,70-74,75-79,80+
Barking and Dagenham,19038,8006,10832,9440,7364,5297,4320,2977,4605
Barnet,39852,18316,20930,19652,17029,14067,13348,9779,14754
Barnsley,22717,10684,16941,16697,14762,12811,12750,9391,11316
Bath and North East Somerset,15428,5316,11823,11871,10277,9125,9637,7276,10663
Bedford,15460,6336,11387,10939,9266,7894,8083,5676,8142


In [102]:
vaccination_rates_by_utla = ((vaccinations_by_utla
    .select_dtypes(exclude='object')
    .div(population_by_utla.select_dtypes(exclude='object')) * 100)
    .combine_first(vaccinations_by_utla)[vaccinations_by_utla.columns])

vaccination_rates_by_utla.loc[:, "Overall"] = vaccinations_by_utla.sum(axis=1).div(population_by_utla.sum(axis=1)) * 100
vaccination_rates_by_utla = vaccination_rates_by_utla.convert_dtypes()

formatting = {
    column: "{:.2f}" 
    for column in vaccination_rates_by_utla.drop(["UTLA Name"], axis=1).columns
}
(vaccination_rates_by_utla
 .sort_values(["Overall"], ascending=True)
 .head(10)
 .style.format(formatting)
 .hide_index())

UTLA Name,Under 45,45-49,50-54,55-59,60-64,65-69,70-74,75-79,80+,Overall
Tower Hamlets,11.90,48.09,72.15,75.11,77.06,79.29,83.54,83.68,84.34,23.87
Hackney,10.42,41.67,64.37,66.09,69.98,76.30,79.28,79.55,77.24,25.10
Newham,11.72,42.49,63.17,66.78,70.74,75.64,79.59,79.91,79.81,25.51
Hammersmith and Fulham,10.94,43.68,64.64,68.12,69.09,73.39,77.45,77.77,78.04,27.35
Islington,11.26,46.74,69.36,72.26,76.39,79.40,83.32,83.89,82.30,27.76
Lambeth,12.13,47.50,63.02,65.79,69.32,72.89,77.16,78.53,77.66,28.01
Westminster,11.49,40.68,56.31,60.36,64.10,67.26,71.14,75.97,77.77,28.23
Southwark,11.66,43.38,66.88,69.50,73.04,75.61,79.60,79.83,79.79,28.29
Wandsworth,12.48,45.80,66.42,69.82,75.11,78.45,82.77,83.61,83.66,28.71
Camden,11.76,44.91,66.26,69.62,74.53,78.28,82.49,85.05,86.02,29.16


In [104]:
population_by_utla.loc[:, "BabyBoomers"] = (population_by_utla[["55-59","60-64", "65-69", "70-74", "75-79", "80+"]].sum(axis=1) / population_by_utla.sum(axis=1)) * 100

formatting = {
    column: "{:,}" 
    for column in population_by_utla.drop(["UTLA Name", "BabyBoomers"], axis=1).columns
}
(population_by_utla
 .sort_values(["BabyBoomers"], ascending=False)
 .head(10)
 .style.format({**formatting, "BabyBoomers": "{:.2f}"})
 .hide_index())

UTLA Name,Under 45,45-49,50-54,55-59,60-64,65-69,70-74,75-79,80+,BabyBoomers
Dorset,"172,648","22,910","27,991","30,719","29,512","27,540","30,850","23,168","32,272",43.77
Isle of Wight,"64,318","8,392","10,765","11,569","10,847","10,614","11,381","8,783","10,710",43.35
East Riding of Yorkshire,"161,761","22,385","26,357","27,873","25,468","23,523","25,784","18,102","24,105",40.76
Torbay,"67,299","8,560","10,481","11,071","10,126","9,422","10,121","7,662","10,434",40.52
Northumberland,"153,750","20,309","23,883","26,342","25,114","22,813","23,136","15,917","20,268",40.29
"Herefordshire, County of","91,990","11,853","14,237","15,117","13,972","12,764","13,193","9,867","13,389",39.86
Shropshire,"154,879","20,359","24,662","25,843","23,185","21,135","22,048","17,102","22,146",39.67
East Sussex,"272,218","35,316","41,780","43,566","39,059","35,635","39,241","29,258","40,870",39.45
Devon,"405,982","49,555","59,192","62,649","58,054","53,527","57,451","43,341","57,948",39.28
Cornwall,"285,145","36,365","42,173","44,714","41,516","38,169","40,938","29,827","37,164",38.98


In [14]:
option = ["Sutton", "South Derbyshire", "Greenwich"]

columns_to_drop = ["Region Code (Administrative)", "Region Name (administrative)", "UTLA Code", "UTLA Name", "LTLA Code"]

local_area = combined.loc[combined["LTLA Name"].isin(option)].drop(columns_to_drop, axis=1)
local_area_absolute = vaccinations.loc[vaccinations["LTLA Name"].isin(option)].drop(columns_to_drop, axis=1)
local_area_population = population.loc[population["LTLA Name"].isin(option)].drop(["LTLA Code"], axis=1)

local_area.rename(columns={"Under 45": "<45"}, inplace=True)
melted_local_area = local_area.melt(value_vars=local_area.columns.drop(["LTLA Name"]), id_vars=["LTLA Name"])
melted_local_area = melted_local_area.rename(columns={"value": "Percentage", "variable": "Age"})    
melted_local_area.reset_index(level=0, inplace=True)

chart = alt.Chart(melted_local_area).mark_bar().encode(
    y=alt.Y('LTLA Name', sort=["index"], axis=alt.Axis(labels=True, ticks=False), title=None),
    x=alt.X('Percentage', scale=alt.Scale(domain=[0, 100])),
    color=alt.Color('LTLA Name', legend=None ),
    row=alt.Row("Age", title=None, sort=["index"]),        
    tooltip=["Age", alt.Tooltip('Percentage', format='.2f')] 
).properties()

chart.save("images/by_local_areas.svg")
chart

alt.Chart(...)

### Vaccination rate by age and local area
<img align="left" src="images/by_local_areas.svg">